生成的QA对主要针对于保单基本定义

In [ ]:
import os
import json
import re
from openai import OpenAI

class CHAT_MODEL:
    def __init__(self, api_key, base_url, model_name):
        self.llm = OpenAI(api_key=api_key, base_url=base_url)
        self.model_name = model_name

    def chat(self, user_prompt):
        completion = self.llm.chat.completions.create(
            model=self.model_name,
            messages=[{"role": "user", "content": user_prompt}],
        )
        return completion.choices[0].message.content


def extract_segments_with_page(md_text, max_segments=5, min_len=50):
    # 使用页码标题分割，如 ## 第3页
    page_splits = re.split(r'#+\s*第\s*(\d+)\s*页', md_text)
    segments = []

    if len(page_splits) <= 1:
        # 如果没有页码标识，按整段分
        raw_paragraphs = re.split(r'\n\s*\n', md_text)
        for para in raw_paragraphs:
            para = para.strip()
            if len(para) >= min_len:
                segments.append(("未知页码", para))
    else:
        for i in range(1, len(page_splits), 2):
            page = page_splits[i]
            content = page_splits[i + 1]
            paragraphs = re.split(r'\n\s*\n', content)
            for para in paragraphs:
                para = para.strip()
                if len(para) >= min_len:
                    segments.append((f"{page}", para))

    total = len(segments)
    if total <= max_segments:
        return segments
    step = total // max_segments
    return [segments[i] for i in range(0, total, step)][:max_segments]


def build_prompt(few_shot_examples, text_segment):
    prompt = "请根据以下文档内容，生成一个问题和答案对，并标明文档页码和原始文本段。\n\n"
    prompt += "【示例1】\n" + few_shot_examples[0] + "\n\n"
    prompt += "【示例2】\n" + few_shot_examples[1] + "\n\n"
    prompt += f"【文档内容】\n{text_segment}\n\n"
    prompt += '''"请生成1个高质量的问答对（问题+答案），问题应基于下列内容，不抄袭标题或目录，必须为用户可能提出的真实业务/操作/流程/条款相关问题。
一、问题部分：为同一个主题创建尽可能多的不同表述的问题，确保问题的多样性。
    每个问题应考虑用户可能的多种问法，如：
        - 什么是...？
        - 是否可以说...？
        - 请解释一下...的含义
        - 如果...会怎样？
        - 能否举个例子说明...？

二、答案部分：提供全面、信息丰富的答案，确保逻辑清晰、准确无误。
请输出JSON格式，字段包括：question, answer, source_file, page_num, content。"'''
    return prompt


def main():
    input_folder = "D:\火力全开的项目实践\宏利 pdf 文件\测试集\data\data"
    output_file = "D:\火力全开的项目实践\宏利 pdf 文件\测试集\data\QA2.json"
    qa_per_doc = 3
    api_key = "2a2299dd95944956b69397a89113d5a7.GW5jR7NYhANOuGES"
    base_url = "https://open.bigmodel.cn/api/paas/v4"
    model_name = "glm-4-0520"

    chat_model = CHAT_MODEL(api_key=api_key, base_url=base_url, model_name=model_name)

    few_shot_examples = [
        '''{
"question": "投资相连寿险保单过了宽限期还没缴费会怎样？",
"answer": "灵活投资宝/万利保障计划/宏宝保单将享有30日宽限期。若过后仍未缴费，客户会收到保单失效通知书，说明如何复效。",
"source_file": "电子行政运作手册_保单行政_2023_10.md",
"page_num": "第6页",
"content": "投資相連壽險計劃保單...將享有30日的寬限期..." }''',
        '''{
"question": "什么情况下保险顾问的营业积分会被扣回？",
"answer": "若某月结日的净已缴保费低于两个月前月结日的净已缴保费，保险顾问的积分将被扣回。",
"source_file": "电子行政运作手册_保单行政_2023_10.md",
"page_num": "第7页",
"content": "於第一年淨保費測試...若（N）月結日的淨已繳保費 ≤（N–2）月結日..." }'''
    ]

    results = []

    for filename in os.listdir(input_folder):
        if not filename.endswith(".md"):
            continue
        file_path = os.path.join(input_folder, filename)
        with open(file_path, "r", encoding="utf-8") as f:
            content = f.read()

        segments = extract_segments_with_page(content, max_segments=qa_per_doc)

        for idx, (page, segment) in enumerate(segments):
            prompt = build_prompt(few_shot_examples, segment)
            try:
                response = chat_model.chat(prompt)

                # 提取 JSON 块
                match = re.search(r'\{[\s\S]*?\}', response)
                if not match:
                    raise ValueError("找不到有效的JSON对象")
                json_str = match.group(0)

                parsed = json.loads(json_str)
                parsed["source_file"] = filename
                parsed["page_num"] = page
                parsed["content"] = segment
                results.append(parsed)

            except Exception as e:
                print(f"错误 - 文件 {filename} 第{idx+1}段（页码：{page}）：{e}")
                continue

    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(results, f, ensure_ascii=False, indent=2)

    print(f"\n成功保存 {len(results)} 个 QA 对到 {output_file}")


if __name__ == "__main__":
    main()


<>:70: SyntaxWarning: invalid escape sequence '\d'
<>:71: SyntaxWarning: invalid escape sequence '\d'
<>:70: SyntaxWarning: invalid escape sequence '\d'
<>:71: SyntaxWarning: invalid escape sequence '\d'
C:\Users\吴芷萱\AppData\Local\Temp\ipykernel_32200\921075933.py:70: SyntaxWarning: invalid escape sequence '\d'
  input_folder = "D:\火力全开的项目实践\宏利 pdf 文件\测试集\data\data"
C:\Users\吴芷萱\AppData\Local\Temp\ipykernel_32200\921075933.py:71: SyntaxWarning: invalid escape sequence '\d'
  output_file = "D:\火力全开的项目实践\宏利 pdf 文件\测试集\data\QA2.json"


错误 - 文件 守护无间危疾保 保单条款_2022_07.md 第2段（页码：16）：Expecting ',' delimiter: line 8 column 2 (char 356)
错误 - 文件 守护无间危疾保 保单条款_2022_07.md 第3段（页码：33）：Invalid \escape: line 6 column 22 (char 214)
错误 - 文件 宏利環球貨幣保障計劃 保單條款_2022_05.md 第3段（页码：12）：Expecting ',' delimiter: line 8 column 3 (char 223)
错误 - 文件 活耀人生危疾保-产品手册.md 第1段（页码：0）：Expecting ',' delimiter: line 8 column 3 (char 283)
错误 - 文件 活耀人生危疾保-产品手册.md 第3段（页码：19）：Unterminated string starting at: line 6 column 12 (char 204)
错误 - 文件 电子行政运作手册_保单行政_2023_10.md 第2段（页码：32）：Expecting ',' delimiter: line 8 column 3 (char 260)
错误 - 文件 电子行政运作手册_理赔_2024_01.md 第1段（页码：0）：Invalid \escape: line 6 column 63 (char 268)

成功保存 14 个 QA 对到 D:\火力全开的项目实践\宏利 pdf 文件\测试集\data\QA2.json
